In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
user_log = pd.read_csv('data_format1/user_log_format1.csv', iterator=True, chunksize=1000)
user_log = pd.concat([chunk[chunk['item_id'] < 160] for chunk in user_log])


In [3]:
train_df = pd.read_csv('./data_format1/train_format1.csv', iterator=True)
test_df = pd.read_csv('./data_format1/test_format1.csv', iterator=True)
user_info = pd.read_csv('./data_format1/user_info_format1.csv', iterator=True)

train_df = pd.concat([chunk[chunk['user_id'].isin(user_log['user_id'])] for chunk in train_df])
test_df = pd.concat([chunk[chunk['user_id'].isin(user_log['user_id'])] for chunk in test_df])
user_info = pd.concat([chunk[chunk['user_id'].isin(user_log['user_id'])] for chunk in user_info])


In [4]:
train_df.rename(columns={'merchant_id': 'seller_id'}, inplace=True)
train_df.head()

test_df.rename(columns={'merchant_id': 'seller_id'}, inplace=True)
test_df.head()

,user_id,seller_id,prob
36,104832,4427,NaN
45,106368,2664,NaN
46,106368,3432,NaN
58,107904,4015,NaN
97,51840,1827,NaN


In [5]:
from sklearn.preprocessing import label_binarize

In [6]:
user_info.isnull().any()

user_id      False
age_range     True
gender        True
dtype: bool

In [7]:
user_info.fillna({'gender': 2, 'age_range': 0}, inplace=True)

user_info.isnull().any()

user_id      False
age_range    False
gender       False
dtype: bool

In [8]:
user_gender = label_binarize(np.array(user_info['gender']), classes=[0,1,2])
user_gender = pd.DataFrame(user_gender, columns=['gender_female','gender_male','gender_unknown'])

user_gender['user_id'] = user_info['user_id'].to_numpy()

user_gender.head()

,gender_female,gender_male,gender_unknown,user_id
0,1,0,0,83378
1,1,0,0,68632
2,1,0,0,1372
3,0,0,1,247820
4,0,1,0,18038


In [9]:
user_age = label_binarize(np.array(user_info['age_range']), classes=list(range(9)))
user_age = pd.DataFrame(user_age, columns=['unknown age', '<18', '[18,24]','[25,29]', '[30,34]', '[35,39]', '[40,49]', '> 50', '> 50'])

user_age['user_id'] = user_info['user_id'].to_numpy()

user_age.head()

,unknown age,<18,"[18,24]","[25,29]","[30,34]","[35,39]","[40,49]",> 50,> 50,user_id
0,0,0,0,0,1,0,0,0,0,83378
1,0,0,0,0,1,0,0,0,0,68632
2,0,0,0,0,0,1,0,0,0,1372
3,0,0,0,0,0,1,0,0,0,247820
4,0,0,0,0,1,0,0,0,0,18038


In [10]:
user_seller_count = user_log.groupby(['user_id','seller_id']).size().reset_index(name='user_seller_interaction')
user_seller_count.head()


,user_id,seller_id,user_seller_interaction
0,28,1381,1
1,28,2781,1
2,69,4053,8
3,114,1289,1
4,138,1381,1


In [11]:
user_actions = user_log.groupby(['user_id', 'action_type']).size().reset_index(name='action_count')

user_actions['user_clicks'] = user_actions[user_actions['action_type'] == 0]['action_count']
user_actions['user_add_to_cart'] = user_actions[user_actions['action_type'] == 1]['action_count']
user_actions['user_purchase'] = user_actions[user_actions['action_type'] == 2]['action_count']
user_actions['user_add_to_favorite'] = user_actions[user_actions['action_type'] == 3]['action_count']

user_actions.fillna(0, inplace=True)
user_actions.drop(columns=['action_type', 'action_count'], inplace=True)
user_actions.head()

,user_id,user_clicks,user_add_to_cart,user_purchase,user_add_to_favorite
0,28,2.0,0.0,0.0,0.0
1,69,7.0,0.0,0.0,0.0
2,69,0.0,0.0,1.0,0.0
3,114,1.0,0.0,0.0,0.0
4,138,1.0,0.0,0.0,0.0


In [12]:
seller_actions = user_log.groupby(['seller_id', 'action_type']).size().reset_index(name='action_count')

seller_actions['seller_clicks'] = seller_actions[seller_actions['action_type'] == 0]['action_count']
seller_actions['seller_add_to_cart'] = seller_actions[seller_actions['action_type'] == 1]['action_count']
seller_actions['seller_purchase'] = seller_actions[seller_actions['action_type'] == 2]['action_count']
seller_actions['seller_add_to_favorite'] = seller_actions[seller_actions['action_type'] == 3]['action_count']

seller_actions.drop(columns=['action_type', 'action_count'], inplace=True)
seller_actions.head()

,seller_id,seller_clicks,seller_add_to_cart,seller_purchase,seller_add_to_favorite
0,1,43.0,NaN,NaN,NaN
1,1,NaN,1.0,NaN,NaN
2,1,NaN,NaN,1.0,NaN
3,1,NaN,NaN,NaN,2.0
4,3,1.0,NaN,NaN,NaN


In [13]:
# only 5 NaN brand / 9k items, negligible
user_log.fillna({'brand_id': 0}, inplace=True)

brand_actions = user_log.groupby(['user_id', 'brand_id', 'action_type']).size().reset_index(name='action_count')

brand_actions['brand_clicks'] = brand_actions[brand_actions['action_type'] == 0]['action_count']
brand_actions['brand_add_to_cart'] = brand_actions[brand_actions['action_type'] == 1]['action_count']
brand_actions['brand_purchase'] = brand_actions[brand_actions['action_type'] == 2]['action_count']
brand_actions['brand_add_to_favorite'] = brand_actions[brand_actions['action_type'] == 3]['action_count']

brand_actions.drop(columns=['brand_id', 'action_type', 'action_count'], inplace=True)
brand_actions.head()

,user_id,brand_clicks,brand_add_to_cart,brand_purchase,brand_add_to_favorite
0,28,1.0,NaN,NaN,NaN
1,28,1.0,NaN,NaN,NaN
2,69,7.0,NaN,NaN,NaN
3,69,NaN,NaN,1.0,NaN
4,114,1.0,NaN,NaN,NaN


In [14]:
user_cat_actions = user_log.groupby(['user_id', 'cat_id', 'action_type']).size().reset_index(name='action_count')

user_cat_actions['cat_clicks'] = user_cat_actions[user_cat_actions['action_type'] == 0]['action_count']
user_cat_actions['cat_add_to_cart'] = user_cat_actions[user_cat_actions['action_type'] == 1]['action_count']
user_cat_actions['cat_purchase'] = user_cat_actions[user_cat_actions['action_type'] == 2]['action_count']
user_cat_actions['cat_add_to_favorite'] = user_cat_actions[user_cat_actions['action_type'] == 3]['action_count']

user_cat_actions.drop(columns=['cat_id', 'action_type', 'action_count'], inplace=True)
user_cat_actions.head()

,user_id,cat_clicks,cat_add_to_cart,cat_purchase,cat_add_to_favorite
0,28,2.0,NaN,NaN,NaN
1,69,7.0,NaN,NaN,NaN
2,69,NaN,NaN,1.0,NaN
3,114,1.0,NaN,NaN,NaN
4,138,1.0,NaN,NaN,NaN


In [15]:
seller_item_count = user_log[['seller_id', 'item_id']].drop_duplicates(['item_id']).groupby('seller_id').size().reset_index(name='seller_item_count')
seller_brand_count = user_log[['seller_id', 'brand_id']].drop_duplicates(['brand_id']).groupby('seller_id').size().reset_index(name='seller_brand_count')
seller_cat_count = user_log[['seller_id', 'cat_id']].drop_duplicates(['cat_id']).groupby('seller_id').size().reset_index(name='seller_cat_count')
seller_total_actions = user_log[['seller_id']].groupby('seller_id').size().reset_index(name='seller_total_actions')

seller_item_count = seller_item_count.merge(seller_brand_count, on=['seller_id'], how='left')
seller_item_count = seller_item_count.merge(seller_cat_count, on=['seller_id'], how='left')
seller_item_count = seller_item_count.merge(seller_total_actions, on=['seller_id'], how='left')

seller_item_count.head()

,seller_id,seller_item_count,seller_brand_count,seller_cat_count,seller_total_actions
0,1,1,1.0,NaN,47
1,3,1,1.0,NaN,1
2,20,1,1.0,NaN,1
3,109,1,1.0,1.0,89
4,112,1,1.0,NaN,1


In [16]:
user_item_actions = user_log[['user_id', 'item_id']].groupby('user_id').size().reset_index(name='user_item_actions')
user_brand_actions = user_log[['user_id', 'brand_id']].groupby('user_id').size().reset_index(name='user_brand_actions')
user_cat_actions = user_log[['user_id', 'cat_id']].groupby('user_id').size().reset_index(name='user_cat_actions')

user_item_actions = user_item_actions.merge(user_brand_actions, on=['user_id'], how='left')
user_item_actions = user_item_actions.merge(user_cat_actions, on=['user_id'], how='left')

user_item_actions.head()

,user_id,user_item_actions,user_brand_actions,user_cat_actions
0,28,2,2,2
1,69,8,8,8
2,114,1,1,1
3,138,1,1,1
4,234,1,1,1


In [17]:
user_seller_actions = user_log[['user_id', 'seller_id', 'item_id']].groupby(['user_id', 'seller_id']).size().reset_index(name='user_seller_actions')
user_seller_brands = user_log[['user_id', 'seller_id', 'brand_id']].groupby(['user_id', 'seller_id']).size().reset_index(name='user_seller_brands')
user_seller_cat = user_log[['user_id', 'seller_id', 'cat_id']].groupby(['user_id', 'seller_id']).size().reset_index(name='user_seller_cat')

user_seller_actions = user_seller_actions.merge(user_seller_brands, on=['user_id', 'seller_id'], how='left')
user_seller_actions = user_seller_actions.merge(user_seller_cat, on=['user_id', 'seller_id'], how='left')

user_seller_actions.head()

,user_id,seller_id,user_seller_actions,user_seller_brands,user_seller_cat
0,28,1381,1,1,1
1,28,2781,1,1,1
2,69,4053,8,8,8
3,114,1289,1,1,1
4,138,1381,1,1,1


In [18]:
user_log['month'] = user_log['time_stamp'] % 100
user_monthly_activity = user_log[['user_id', 'month']].drop_duplicates(['month']).groupby('user_id').size().reset_index(name='user_monthly_activity')
seller_monthly_activity = user_log[['user_id', 'seller_id', 'month']].drop_duplicates(['month']).groupby(['user_id', 'seller_id']).size().reset_index(name='seller_monthly_activity')

user_daily_activity = user_log[['user_id', 'time_stamp']].drop_duplicates(['time_stamp']).groupby('user_id').size().reset_index(name='user_daily_activity')
seller_daily_activity = user_log[['user_id', 'seller_id', 'time_stamp']].drop_duplicates(['time_stamp']).groupby(['user_id', 'seller_id']).size().reset_index(name='seller_daily_activity')

seller_monthly_activity = seller_monthly_activity.merge(user_monthly_activity, on=['user_id'], how='left')
seller_monthly_activity = seller_monthly_activity.merge(user_daily_activity, on=['user_id'], how='left')
seller_monthly_activity = seller_monthly_activity.merge(seller_daily_activity, on=['user_id', 'seller_id'], how='left')

seller_monthly_activity.head()

,user_id,seller_id,seller_monthly_activity,user_monthly_activity,user_daily_activity,seller_daily_activity
0,549,1644,1,1,1,1
1,3792,3695,1,1,1,1
2,28690,3484,1,1,2,1
3,34436,109,1,1,1,1
4,58013,1835,1,1,1,1


In [19]:

# merge

def merge(x):
    x = x.merge(user_age, on=['user_id'], how='left')
    x = x.merge(user_gender, on=['user_id'], how='left')

    x = x.merge(user_seller_count, on=['user_id', 'seller_id'], how='left')
    x = x.merge(user_actions, on=['user_id'], how='left')
    x = x.merge(seller_actions, on=['seller_id'], how='left')

    x = x.merge(brand_actions, on=['user_id'], how='left')
    x = x.merge(user_cat_actions, on=['user_id'], how='left')
    x = x.merge(seller_item_count, on=['seller_id'], how='left')

    x = x.merge(user_item_actions, on=['user_id'], how='left')

    x = x.merge(user_seller_actions, on=['user_id', 'seller_id'], how='left')
    x = x.merge(seller_monthly_activity, on=['user_id', 'seller_id'], how='left')
    
    x.drop('user_id', axis=1)
    x.drop('seller_id', axis=1)

    x.fillna(0, inplace=True)
    return x

train_df = merge(train_df)
test_df = merge(test_df)
list(train_df.columns)


['user_id',
 'seller_id',
 'label',
 'unknown age',
 '<18',
 '[18,24]',
 '[25,29]',
 '[30,34]',
 '[35,39]',
 '[40,49]',
 '> 50',
 '> 50',
 'gender_female',
 'gender_male',
 'gender_unknown',
 'user_seller_interaction',
 'user_clicks',
 'user_add_to_cart',
 'user_purchase',
 'user_add_to_favorite',
 'seller_clicks',
 'seller_add_to_cart',
 'seller_purchase',
 'seller_add_to_favorite',
 'brand_clicks',
 'brand_add_to_cart',
 'brand_purchase',
 'brand_add_to_favorite',
 'user_cat_actions_x',
 'seller_item_count',
 'seller_brand_count',
 'seller_cat_count',
 'seller_total_actions',
 'user_item_actions',
 'user_brand_actions',
 'user_cat_actions_y',
 'user_seller_actions',
 'user_seller_brands',
 'user_seller_cat',
 'seller_monthly_activity',
 'user_monthly_activity',
 'user_daily_activity',
 'seller_daily_activity']

In [20]:
train_df.to_csv('train_df.csv')

test_df.to_csv('test_df.csv')
